# Homo-NN Tutorial

The Homo(horizontal) federated learning allows parties to collaboratively train a neural network model without sharing their actual data. In a horizontally partitioned data setting, multiple parties have the same feature set but different user samples. In this scenario, each party trains the model locally on its own subset of data and only shares the model updates.

The Homo-NN algorithm is designed for horizontal federated neural network training. In this tutorial, we demonstrate how to run a Homo-NN task under FATE-2.0 without using a Pipeline. This is particularly useful for local experimentation, model/training setting modifications and testing. However, it is not recommended for direct usage in a production environment. 

## Setup Homo-NN Setup by Step

To run a Hetero-Secureboost task, several steps are needed:
1. Import required libraries and create fate context
2. Prepare datasets, models, loss and optimizers
3. Configure clients(guest&hosts) parameters, trainers; configure arbiter parameters and trainer
4. Run the training script

### Import Libs and Create Context
We import these lib from later use. The FedAVGCLient is the trainer on the client sides and the FedAVGServer is the trainer on the server side. In TrainingArgument we can set training parameters(the same as the transformer trainer) while in the FeAVGArguments we can adjust options for federation.

If fate is not installed, run:
```
pip install fate==2.0.0b0
```

In [1]:
from fate.ml.nn.algo.homo.fedavg import FedAVGCLient, FedAVGServer, TrainingArguments, FedAVGArguments
import torch as t
import pandas as pd
from fate.ml.nn.dataset.table import TableDataset
import sys
from datetime import datetime

We will use 'create_ctx' to create fate context. When creating fate context, please make sure that every party's context is initialized with the same unique context_name. Here, we provide 'get_current_datetime_str' in order to get a unique context name
according to the current date and time. We are running a three party homo-federation task, so parties are:

In [2]:
arbiter = ("arbiter", 10000)
guest = ("guest", 10000)
host = ("host", 9999)

In [3]:
def get_current_datetime_str():
    return datetime.now().strftime("%Y-%m-%d-%H-%M")


def create_ctx(local, context_name):
    from fate.arch import Context
    from fate.arch.computing.standalone import CSession
    from fate.arch.federation.standalone import StandaloneFederation

    # init fate context
    computing = CSession()
    return Context(
        computing=computing, federation=StandaloneFederation(computing, context_name, local, [guest, host])
    )

## Prepare

Using FATE's Homo-NN framework is very similar to directly using PyTorch's nn.Module, Dataset, and Optimizer, as well as Transformers. Developed based on HuggingFace's Transformers Trainer, FATE-2.0 NN Trainer enables seamless integration for a variety of neural network tasks. Whether you're dealing with tabular data, natural language processing (NLP), or computer vision (CV), our Homo-NN framework easily adapts to PyTorch's components. Therefore, you can prepare your project just as you would when using PyTorch's Module, Dataset, and Optimizer. Here, we adopt a binary classification task as the example.

### Read Csv and Prepare Dataset

We read the csv data of guest side, and put it into the TensorDataset.

In [4]:
guest_data_path = './../FATE/examples/data/breast_homo_guest.csv'

In [5]:
import torch
from torch.utils.data import TensorDataset

idx = 'id'
label_name = 'y'
guest_data = pd.read_csv(guest_data_path, index_col=idx)

X = guest_data.drop(columns=[label_name])
y = guest_data.pop(label_name).values

X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).view(-1, 1)
dataset = TensorDataset(X, y)

Or you can use our built-in TableDataset

In [6]:
from fate.ml.nn.dataset.table import TableDataset

dataset_ = TableDataset(to_tensor=True)
dataset_.load(guest_data_path)

In [7]:
dataset[0]

(tensor([ 0.2549, -1.0466,  0.2097,  0.0742, -0.4414, -0.3776, -0.4859,  0.3471,
         -0.2876, -0.7335,  0.4495, -1.2472,  0.4132,  0.3038, -0.1238, -0.1842,
         -0.2191,  0.2685,  0.0160, -0.7893, -0.3374, -0.7282, -0.4426, -0.2728,
         -0.6080, -0.5772, -0.5011,  0.1434, -0.4664, -0.5541]),
 tensor([1.]))

In [8]:
dataset[0][0].shape

torch.Size([30])

### Prepare Model, Loss and Optimizer

We initialize the pytorch model, loss and optimizer.

In [9]:
import torch

model = torch.nn.Sequential(
    torch.nn.Linear(30, 16),
    torch.nn.ReLU(),
    torch.nn.Linear(16, 1),
    torch.nn.Sigmoid()
)

loss_fn = torch.nn.BCELoss()
opt = torch.optim.Adam(model.parameters(), lr=0.01)

### Run Trainer without Federation

Here, we first perform a non-federated local training using FedAVGClient to verify whether our model, dataset, optimizer, and loss function can work properly. We can enable FedAVG to operate in local mode by calling 'set_local_mode()'.


In [16]:
ctx = create_ctx(guest, get_current_datetime_str())  # create a guest context for local testing
train_arg = TrainingArguments(num_train_epochs=4, per_device_train_batch_size=128, disable_tqdm=False)
trainer = FedAVGCLient(ctx, model, train_arg, FedAVGArguments(), train_set=dataset, loss_fn=loss_fn, optimizer=opt)
trainer.set_local_mode()

In [17]:
trainer.train()

Step,Training Loss
2,0.067400
4,0.060600
6,0.053600
8,0.049600


TrainOutput(global_step=8, training_loss=0.057804910466074944, metrics={'train_runtime': 0.0137, 'train_samples_per_second': 66308.488, 'train_steps_per_second': 584.216, 'train_loss': 0.057804910466074944, 'epoch': 4.0})

Excellent! Everything is functioning as expected. Now, we can proceed to develop a script that contains the code for both the clients (guest & host) and the server (arbiter).

 Once the script is ready, we can execute our task using three separate terminals.

### Run the Homo-NN Script

Once model, dataset, optimizer and loss are prepared, we can initialize FedAVGClient instances and fit models. The server side is responsible for model aggreation, so we only need to intialize a FedAVGServer on arbiter side, and call its train function.

**Please note** that when configuring `FedAVGArguments`, **make sure that the number of aggregation rounds is consistent across all parties**. We will **check the aggregation round count**, and **if there is any inconsistency, the task will fail**.

Here is the complete python script:

In [ ]:
from fate.ml.nn.algo.homo.fedavg import FedAVGCLient, FedArguments, TrainingArguments, FedAVGServer
import torch as t
import pandas as pd
from fate.ml.nn.dataset.table import TableDataset
import sys
from datetime import datetime


arbiter = ("arbiter", 10000)
guest = ("guest", 10000)
host = ("host", 9999)

def get_current_datetime_str():
    return datetime.now().strftime("%Y-%m-%d-%H-%M")


def create_ctx(local):
    from fate.arch import Context
    from fate.arch.computing.standalone import CSession
    from fate.arch.federation.standalone import StandaloneFederation
    import logging

    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)

    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.DEBUG)

    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    console_handler.setFormatter(formatter)

    logger.addHandler(console_handler)
    computing = CSession()
    return Context(
        computing=computing, federation=StandaloneFederation(computing, get_current_datetime_str(), local, [guest, host, arbiter])
    )


if __name__ == "__main__":

    model = t.nn.Sequential(t.nn.Linear(30, 1), t.nn.Sigmoid())

    if sys.argv[1] == "guest":

        ds = TableDataset(return_dict=False, to_tensor=True)
        ds.load("../../../../../../examples/data/breast_homo_guest.csv")
        ctx = create_ctx(guest)
        fed_args = FedArguments(aggregate_strategy="epochs", aggregate_freq=1, aggregator="secure_aggregate")
        args = TrainingArguments(
            num_train_epochs=5, per_device_train_batch_size=16, logging_strategy="steps", logging_steps=5
        )
        trainer = FedAVGCLient(
            ctx=ctx,
            model=model,
            fed_args=fed_args,
            training_args=args,
            loss_fn=t.nn.BCELoss(),
            optimizer=t.optim.SGD(model.parameters(), lr=0.01),
            train_set=ds,
        )
        trainer.train()

    elif sys.argv[1] == "host":

        ds = TableDataset(return_dict=False, to_tensor=True)
        ds.load("../../../../../../examples/data/breast_homo_host.csv")
        ctx = create_ctx(host)
        fed_args = FedArguments(aggregate_strategy="epochs", aggregate_freq=1, aggregator="secure_aggregate")
        args = TrainingArguments(num_train_epochs=5, per_device_train_batch_size=16)
        trainer = FedAVGCLient(
            ctx=ctx,
            model=model,
            fed_args=fed_args,
            training_args=args,
            loss_fn=t.nn.BCELoss(),
            optimizer=t.optim.SGD(model.parameters(), lr=0.01),
            train_set=ds,
        )
        trainer.train()

    else:
        ctx = create_ctx(arbiter)
        trainer = FedAVGServer(ctx)
        trainer.train()

We save this script to a file named 'run_homo_nn.py' and execute it simultaneously in three terminals. The guest party & host party terminals' commands is:

```
python -i ./run_homo_nn.py guest
```

```
python -i ./run_homo_nn.py host
```


The host party terminal command is:

```
python -i ./run_homo_nn.py arbiter
```

We add -i option so that you can check the result of the script in the terminal.

Guest Terminal Outputs:

```
2023-09-13 20:36:13,319 - fate.arch.federation.standalone._federation - DEBUG - [federation.standalone]init federation: standalone_session=<fate.arch.computing.standalone._csession.CSession object at 0x7f90881e3a60>, federation_session_id=2023-09-13-20-36, party=('guest', 10000)
2023-09-13 20:36:13,319 - fate.arch.federation.standalone._federation - DEBUG - [federation.standalone]init federation context done
2023-09-13 20:36:13,400 - fate.ml.nn.algo.homo.fedavg - INFO - Using secure_aggregate aggregator
2023-09-13 20:36:13,400 - fate.arch._standalone - DEBUG - [federation.standalone.remote.agg_type.default]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,400 - fate.arch._standalone - DEBUG - [federation.standalone.remote.agg_type.default]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,413 - fate.ml.aggregator.base - INFO - computing weights
2023-09-13 20:36:13,413 - fate.arch._standalone - DEBUG - [federation.standalone.remote.local_weight_fedavg_0.default]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,413 - fate.arch._standalone - DEBUG - [federation.standalone.remote.local_weight_fedavg_0.default]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,428 - fate.arch._standalone - DEBUG - [federation.standalone.get.agg_weight_fedavg_0.default]
2023-09-13 20:36:13,529 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-agg_weight_fedavg_0-default-arbiter-10000-guest-10000 type Object
2023-09-13 20:36:13,536 - fate.arch._standalone - DEBUG - [federation.standalone.get.agg_weight_fedavg_0.default] got object with type: <class 'bytes'>
2023-09-13 20:36:13,545 - fate.ml.aggregator.base - INFO - aggregate weight is 0.4989010989010989
2023-09-13 20:36:13,545 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_dh_pubkey.default]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,545 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_dh_pubkey.default]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,556 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_dh_pubkey.default]
2023-09-13 20:36:13,556 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_dh_pubkey-default-host-9999-guest-10000 type Object
2023-09-13 20:36:13,561 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_dh_pubkey.default] got object with type: <class 'bytes'>
2023-09-13 20:36:13,566 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_loss_dh_pubkey.default]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,566 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_loss_dh_pubkey.default]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,576 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_loss_dh_pubkey.default]
2023-09-13 20:36:13,677 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_loss_dh_pubkey-default-host-9999-guest-10000 type Object
2023-09-13 20:36:13,683 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_loss_dh_pubkey.default] got object with type: <class 'bytes'>
2023-09-13 20:36:13,690 - fate.ml.nn.trainer.trainer_base - INFO - computed max_aggregation is 5
2023-09-13 20:36:13,690 - fate.ml.nn.trainer.trainer_base - INFO - parameters is {'num_train_epochs': 5, 'max_steps': 75, 'num_update_steps_per_epoch': 15, 'epochs_trained': 0, 'steps_trained_in_current_epoch': 0, 'max_aggregation': 5, 'aggregate_freq': 1, 'aggregation_strategy': 'epochs'}
2023-09-13 20:36:13,690 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fed_para_0.default]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,690 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fed_para_0.default]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,722 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.7378, 'learning_rate': 0.01, 'epoch': 0.33}
2023-09-13 20:36:13,729 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.741, 'learning_rate': 0.01, 'epoch': 0.67}
2023-09-13 20:36:13,736 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.6012, 'learning_rate': 0.01, 'epoch': 1.0}
2023-09-13 20:36:13,736 - fate.ml.nn.trainer.trainer_base - INFO - aggregation on epoch end
2023-09-13 20:36:13,737 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-0]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,737 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-0]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,746 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-0]
2023-09-13 20:36:13,846 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_aggregated_values-default.aggregation-0-arbiter-10000-guest-10000 type Object
2023-09-13 20:36:13,851 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-0] got object with type: <class 'bytes'>
2023-09-13 20:36:13,856 - fate.ml.nn.trainer.trainer_base - INFO - Aggregation count: 1 / 5
2023-09-13 20:36:13,863 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.5073, 'learning_rate': 0.01, 'epoch': 1.33}
2023-09-13 20:36:13,871 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.4734, 'learning_rate': 0.01, 'epoch': 1.67}
2023-09-13 20:36:13,877 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.4241, 'learning_rate': 0.01, 'epoch': 2.0}
2023-09-13 20:36:13,878 - fate.ml.nn.trainer.trainer_base - INFO - aggregation on epoch end
2023-09-13 20:36:13,878 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-1]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,878 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-1]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,888 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-1]
2023-09-13 20:36:13,989 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_aggregated_values-default.aggregation-1-arbiter-10000-guest-10000 type Object
2023-09-13 20:36:13,993 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-1] got object with type: <class 'bytes'>
2023-09-13 20:36:13,998 - fate.ml.nn.trainer.trainer_base - INFO - Aggregation count: 2 / 5
2023-09-13 20:36:14,006 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.3753, 'learning_rate': 0.01, 'epoch': 2.33}
2023-09-13 20:36:14,014 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.4512, 'learning_rate': 0.01, 'epoch': 2.67}
2023-09-13 20:36:14,021 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.4011, 'learning_rate': 0.01, 'epoch': 3.0}
2023-09-13 20:36:14,021 - fate.ml.nn.trainer.trainer_base - INFO - aggregation on epoch end
2023-09-13 20:36:14,022 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-2]remote data, type=<class 'bytes'>
2023-09-13 20:36:14,022 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-2]remote object with type: <class 'bytes'>
2023-09-13 20:36:14,031 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-2]
2023-09-13 20:36:14,131 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_aggregated_values-default.aggregation-2-arbiter-10000-guest-10000 type Object
2023-09-13 20:36:14,136 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-2] got object with type: <class 'bytes'>
2023-09-13 20:36:14,140 - fate.ml.nn.trainer.trainer_base - INFO - Aggregation count: 3 / 5
2023-09-13 20:36:14,148 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.3772, 'learning_rate': 0.01, 'epoch': 3.33}
2023-09-13 20:36:14,155 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.3531, 'learning_rate': 0.01, 'epoch': 3.67}
2023-09-13 20:36:14,162 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.338, 'learning_rate': 0.01, 'epoch': 4.0}
2023-09-13 20:36:14,162 - fate.ml.nn.trainer.trainer_base - INFO - aggregation on epoch end
2023-09-13 20:36:14,162 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-3]remote data, type=<class 'bytes'>
2023-09-13 20:36:14,162 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-3]remote object with type: <class 'bytes'>
2023-09-13 20:36:14,171 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-3]
2023-09-13 20:36:14,272 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_aggregated_values-default.aggregation-3-arbiter-10000-guest-10000 type Object
2023-09-13 20:36:14,277 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-3] got object with type: <class 'bytes'>
2023-09-13 20:36:14,282 - fate.ml.nn.trainer.trainer_base - INFO - Aggregation count: 4 / 5
2023-09-13 20:36:14,290 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.32, 'learning_rate': 0.01, 'epoch': 4.33}
2023-09-13 20:36:14,297 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.3118, 'learning_rate': 0.01, 'epoch': 4.67}
2023-09-13 20:36:14,303 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.3319, 'learning_rate': 0.01, 'epoch': 5.0}
2023-09-13 20:36:14,304 - fate.ml.nn.trainer.trainer_base - INFO - aggregation on epoch end
2023-09-13 20:36:14,304 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-4]remote data, type=<class 'bytes'>
2023-09-13 20:36:14,304 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-4]remote object with type: <class 'bytes'>
2023-09-13 20:36:14,314 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-4]
2023-09-13 20:36:14,415 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_aggregated_values-default.aggregation-4-arbiter-10000-guest-10000 type Object
2023-09-13 20:36:14,419 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-4] got object with type: <class 'bytes'>
2023-09-13 20:36:14,424 - fate.ml.nn.trainer.trainer_base - INFO - Aggregation count: 5 / 5
2023-09-13 20:36:14,424 - fate.ml.nn.trainer.trainer_base - INFO - {'train_runtime': 1.024, 'train_samples_per_second': 1108.351, 'train_steps_per_second': 73.239, 'train_loss': 0.4496373208363851, 'epoch': 5.0}
```

Host Terminal Outputs:

```
2023-09-13 20:36:12,803 - fate.arch.federation.standalone._federation - DEBUG - [federation.standalone]init federation: standalone_session=<fate.arch.computing.standalone._csession.CSession object at 0x7fcd12dc2ee0>, federation_session_id=2023-09-13-20-36, party=('host', 9999)
2023-09-13 20:36:12,803 - fate.arch.federation.standalone._federation - DEBUG - [federation.standalone]init federation context done
2023-09-13 20:36:12,888 - fate.ml.nn.algo.homo.fedavg - INFO - Using secure_aggregate aggregator
2023-09-13 20:36:12,888 - fate.arch._standalone - DEBUG - [federation.standalone.remote.agg_type.default]remote data, type=<class 'bytes'>
2023-09-13 20:36:12,888 - fate.arch._standalone - DEBUG - [federation.standalone.remote.agg_type.default]remote object with type: <class 'bytes'>
2023-09-13 20:36:12,899 - fate.ml.aggregator.base - INFO - computing weights
2023-09-13 20:36:12,899 - fate.arch._standalone - DEBUG - [federation.standalone.remote.local_weight_fedavg_0.default]remote data, type=<class 'bytes'>
2023-09-13 20:36:12,899 - fate.arch._standalone - DEBUG - [federation.standalone.remote.local_weight_fedavg_0.default]remote object with type: <class 'bytes'>
2023-09-13 20:36:12,909 - fate.arch._standalone - DEBUG - [federation.standalone.get.agg_weight_fedavg_0.default]
2023-09-13 20:36:13,512 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-agg_weight_fedavg_0-default-arbiter-10000-host-9999 type Object
2023-09-13 20:36:13,518 - fate.arch._standalone - DEBUG - [federation.standalone.get.agg_weight_fedavg_0.default] got object with type: <class 'bytes'>
2023-09-13 20:36:13,523 - fate.ml.aggregator.base - INFO - aggregate weight is 0.5010989010989011
2023-09-13 20:36:13,524 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_dh_pubkey.default]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,524 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_dh_pubkey.default]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,539 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_dh_pubkey.default]
2023-09-13 20:36:13,640 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_dh_pubkey-default-guest-10000-host-9999 type Object
2023-09-13 20:36:13,646 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_dh_pubkey.default] got object with type: <class 'bytes'>
2023-09-13 20:36:13,652 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_loss_dh_pubkey.default]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,653 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_loss_dh_pubkey.default]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,661 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_loss_dh_pubkey.default]
2023-09-13 20:36:13,661 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_loss_dh_pubkey-default-guest-10000-host-9999 type Object
2023-09-13 20:36:13,666 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_loss_dh_pubkey.default] got object with type: <class 'bytes'>
2023-09-13 20:36:13,671 - fate.ml.nn.trainer.trainer_base - INFO - computed max_aggregation is 5
2023-09-13 20:36:13,671 - fate.ml.nn.trainer.trainer_base - INFO - parameters is {'num_train_epochs': 5, 'max_steps': 75, 'num_update_steps_per_epoch': 15, 'epochs_trained': 0, 'steps_trained_in_current_epoch': 0, 'max_aggregation': 5, 'aggregate_freq': 1, 'aggregation_strategy': 'epochs'}
2023-09-13 20:36:13,671 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fed_para_0.default]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,671 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fed_para_0.default]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,688 - fate.ml.nn.trainer.trainer_base - INFO - aggregation on epoch end
2023-09-13 20:36:13,689 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-0]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,689 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-0]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,708 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-0]
2023-09-13 20:36:13,909 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_aggregated_values-default.aggregation-0-arbiter-10000-host-9999 type Object
2023-09-13 20:36:13,914 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-0] got object with type: <class 'bytes'>
2023-09-13 20:36:13,919 - fate.ml.nn.trainer.trainer_base - INFO - Aggregation count: 1 / 5
2023-09-13 20:36:13,919 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.5443, 'learning_rate': 0.01, 'epoch': 1.0}
2023-09-13 20:36:13,940 - fate.ml.nn.trainer.trainer_base - INFO - aggregation on epoch end
2023-09-13 20:36:13,940 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-1]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,941 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-1]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,949 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-1]
2023-09-13 20:36:14,050 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_aggregated_values-default.aggregation-1-arbiter-10000-host-9999 type Object
2023-09-13 20:36:14,055 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-1] got object with type: <class 'bytes'>
2023-09-13 20:36:14,060 - fate.ml.nn.trainer.trainer_base - INFO - Aggregation count: 2 / 5
2023-09-13 20:36:14,060 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.4891, 'learning_rate': 0.01, 'epoch': 2.0}
2023-09-13 20:36:14,081 - fate.ml.nn.trainer.trainer_base - INFO - aggregation on epoch end
2023-09-13 20:36:14,081 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-2]remote data, type=<class 'bytes'>
2023-09-13 20:36:14,081 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-2]remote object with type: <class 'bytes'>
2023-09-13 20:36:14,090 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-2]
2023-09-13 20:36:14,191 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_aggregated_values-default.aggregation-2-arbiter-10000-host-9999 type Object
2023-09-13 20:36:14,195 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-2] got object with type: <class 'bytes'>
2023-09-13 20:36:14,200 - fate.ml.nn.trainer.trainer_base - INFO - Aggregation count: 3 / 5
2023-09-13 20:36:14,200 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.4143, 'learning_rate': 0.01, 'epoch': 3.0}
2023-09-13 20:36:14,221 - fate.ml.nn.trainer.trainer_base - INFO - aggregation on epoch end
2023-09-13 20:36:14,222 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-3]remote data, type=<class 'bytes'>
2023-09-13 20:36:14,222 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-3]remote object with type: <class 'bytes'>
2023-09-13 20:36:14,234 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-3]
2023-09-13 20:36:14,334 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_aggregated_values-default.aggregation-3-arbiter-10000-host-9999 type Object
2023-09-13 20:36:14,339 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-3] got object with type: <class 'bytes'>
2023-09-13 20:36:14,344 - fate.ml.nn.trainer.trainer_base - INFO - Aggregation count: 4 / 5
2023-09-13 20:36:14,344 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.3502, 'learning_rate': 0.01, 'epoch': 4.0}
2023-09-13 20:36:14,365 - fate.ml.nn.trainer.trainer_base - INFO - aggregation on epoch end
2023-09-13 20:36:14,365 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-4]remote data, type=<class 'bytes'>
2023-09-13 20:36:14,365 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_mixed_client_values.default.aggregation-4]remote object with type: <class 'bytes'>
2023-09-13 20:36:14,375 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-4]
2023-09-13 20:36:14,476 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_aggregated_values-default.aggregation-4-arbiter-10000-host-9999 type Object
2023-09-13 20:36:14,480 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_aggregated_values.default.aggregation-4] got object with type: <class 'bytes'>
2023-09-13 20:36:14,485 - fate.ml.nn.trainer.trainer_base - INFO - Aggregation count: 5 / 5
2023-09-13 20:36:14,486 - fate.ml.nn.trainer.trainer_base - INFO - {'loss': 0.3157, 'learning_rate': 0.01, 'epoch': 5.0}
2023-09-13 20:36:14,486 - fate.ml.nn.trainer.trainer_base - INFO - {'train_runtime': 1.5982, 'train_samples_per_second': 713.303, 'train_steps_per_second': 46.928, 'train_loss': 0.4227139218648275, 'epoch': 5.0}
```

Arbiter Terminal Outputs:

```
2023-09-13 20:36:12,315 - fate.arch.federation.standalone._federation - DEBUG - [federation.standalone]init federation: standalone_session=<fate.arch.computing.standalone._csession.CSession object at 0x7fb2f65c1dc0>, federation_session_id=2023-09-13-20-36, party=('arbiter', 10000)
2023-09-13 20:36:12,316 - fate.arch.federation.standalone._federation - DEBUG - [federation.standalone]init federation context done
2023-09-13 20:36:12,316 - fate.arch._standalone - DEBUG - [federation.standalone.get.agg_type.default]
2023-09-13 20:36:13,418 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-agg_type-default-guest-10000-arbiter-10000 type Object
2023-09-13 20:36:13,425 - fate.arch._standalone - DEBUG - [federation.standalone.get.agg_type.default] got object with type: <class 'bytes'>
2023-09-13 20:36:13,432 - fate.arch._standalone - DEBUG - [federation.standalone.get.agg_type.default]
2023-09-13 20:36:13,432 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-agg_type-default-host-9999-arbiter-10000 type Object
2023-09-13 20:36:13,436 - fate.arch._standalone - DEBUG - [federation.standalone.get.agg_type.default] got object with type: <class 'bytes'>
2023-09-13 20:36:13,441 - fate.ml.nn.algo.homo.fedavg - INFO - Using secure_aggregate aggregator
2023-09-13 20:36:13,441 - fate.arch._standalone - DEBUG - [federation.standalone.get.local_weight_fedavg_0.default]
2023-09-13 20:36:13,441 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-local_weight_fedavg_0-default-guest-10000-arbiter-10000 type Object
2023-09-13 20:36:13,446 - fate.arch._standalone - DEBUG - [federation.standalone.get.local_weight_fedavg_0.default] got object with type: <class 'bytes'>
2023-09-13 20:36:13,450 - fate.arch._standalone - DEBUG - [federation.standalone.get.local_weight_fedavg_0.default]
2023-09-13 20:36:13,450 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-local_weight_fedavg_0-default-host-9999-arbiter-10000 type Object
2023-09-13 20:36:13,454 - fate.arch._standalone - DEBUG - [federation.standalone.get.local_weight_fedavg_0.default] got object with type: <class 'bytes'>
2023-09-13 20:36:13,459 - fate.arch._standalone - DEBUG - [federation.standalone.remote.agg_weight_fedavg_0.default]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,459 - fate.arch._standalone - DEBUG - [federation.standalone.remote.agg_weight_fedavg_0.default]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,470 - fate.arch._standalone - DEBUG - [federation.standalone.remote.agg_weight_fedavg_0.default]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,470 - fate.arch._standalone - DEBUG - [federation.standalone.remote.agg_weight_fedavg_0.default]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,482 - fate.ml.nn.trainer.trainer_base - INFO - Initialized aggregator Done: <fate.ml.aggregator.secure_aggregator.SecureAggregatorServer object at 0x7fb2f65b0520>
2023-09-13 20:36:13,482 - fate.arch._standalone - DEBUG - [federation.standalone.get.fed_para_0.default]
2023-09-13 20:36:13,682 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fed_para_0-default-host-9999-arbiter-10000 type Object
2023-09-13 20:36:13,692 - fate.arch._standalone - DEBUG - [federation.standalone.get.fed_para_0.default] got object with type: <class 'bytes'>
2023-09-13 20:36:13,700 - fate.arch._standalone - DEBUG - [federation.standalone.get.fed_para_0.default]
2023-09-13 20:36:13,800 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fed_para_0-default-guest-10000-arbiter-10000 type Object
2023-09-13 20:36:13,805 - fate.arch._standalone - DEBUG - [federation.standalone.get.fed_para_0.default] got object with type: <class 'bytes'>
2023-09-13 20:36:13,809 - fate.ml.nn.trainer.trainer_base - INFO - checked parameters are {'max_aggregation': 5}
2023-09-13 20:36:13,810 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-0]
2023-09-13 20:36:13,810 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_mixed_client_values-default.aggregation-0-guest-10000-arbiter-10000 type Object
2023-09-13 20:36:13,815 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-0] got object with type: <class 'bytes'>
2023-09-13 20:36:13,819 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-0]
2023-09-13 20:36:13,819 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_mixed_client_values-default.aggregation-0-host-9999-arbiter-10000 type Object
2023-09-13 20:36:13,824 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-0] got object with type: <class 'bytes'>
2023-09-13 20:36:13,829 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-0]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,829 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-0]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,838 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-0]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,838 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-0]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,847 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-1]
2023-09-13 20:36:13,947 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_mixed_client_values-default.aggregation-1-guest-10000-arbiter-10000 type Object
2023-09-13 20:36:13,954 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-1] got object with type: <class 'bytes'>
2023-09-13 20:36:13,958 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-1]
2023-09-13 20:36:13,958 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_mixed_client_values-default.aggregation-1-host-9999-arbiter-10000 type Object
2023-09-13 20:36:13,963 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-1] got object with type: <class 'bytes'>
2023-09-13 20:36:13,968 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-1]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,968 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-1]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,977 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-1]remote data, type=<class 'bytes'>
2023-09-13 20:36:13,977 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-1]remote object with type: <class 'bytes'>
2023-09-13 20:36:13,986 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-2]
2023-09-13 20:36:14,086 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_mixed_client_values-default.aggregation-2-guest-10000-arbiter-10000 type Object
2023-09-13 20:36:14,094 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-2] got object with type: <class 'bytes'>
2023-09-13 20:36:14,098 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-2]
2023-09-13 20:36:14,098 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_mixed_client_values-default.aggregation-2-host-9999-arbiter-10000 type Object
2023-09-13 20:36:14,103 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-2] got object with type: <class 'bytes'>
2023-09-13 20:36:14,107 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-2]remote data, type=<class 'bytes'>
2023-09-13 20:36:14,107 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-2]remote object with type: <class 'bytes'>
2023-09-13 20:36:14,116 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-2]remote data, type=<class 'bytes'>
2023-09-13 20:36:14,116 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-2]remote object with type: <class 'bytes'>
2023-09-13 20:36:14,125 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-3]
2023-09-13 20:36:14,225 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_mixed_client_values-default.aggregation-3-guest-10000-arbiter-10000 type Object
2023-09-13 20:36:14,232 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-3] got object with type: <class 'bytes'>
2023-09-13 20:36:14,239 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-3]
2023-09-13 20:36:14,239 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_mixed_client_values-default.aggregation-3-host-9999-arbiter-10000 type Object
2023-09-13 20:36:14,243 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-3] got object with type: <class 'bytes'>
2023-09-13 20:36:14,247 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-3]remote data, type=<class 'bytes'>
2023-09-13 20:36:14,248 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-3]remote object with type: <class 'bytes'>
2023-09-13 20:36:14,257 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-3]remote data, type=<class 'bytes'>
2023-09-13 20:36:14,257 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-3]remote object with type: <class 'bytes'>
2023-09-13 20:36:14,266 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-4]
2023-09-13 20:36:14,366 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_mixed_client_values-default.aggregation-4-guest-10000-arbiter-10000 type Object
2023-09-13 20:36:14,377 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-4] got object with type: <class 'bytes'>
2023-09-13 20:36:14,382 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-4]
2023-09-13 20:36:14,382 - fate.arch._standalone - DEBUG - [GET] Got 2023-09-13-20-36-fedavg_model_mixed_client_values-default.aggregation-4-host-9999-arbiter-10000 type Object
2023-09-13 20:36:14,386 - fate.arch._standalone - DEBUG - [federation.standalone.get.fedavg_model_mixed_client_values.default.aggregation-4] got object with type: <class 'bytes'>
2023-09-13 20:36:14,391 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-4]remote data, type=<class 'bytes'>
2023-09-13 20:36:14,391 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-4]remote object with type: <class 'bytes'>
2023-09-13 20:36:14,400 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-4]remote data, type=<class 'bytes'>
2023-09-13 20:36:14,400 - fate.arch._standalone - DEBUG - [federation.standalone.remote.fedavg_model_aggregated_values.default.aggregation-4]remote object with type: <class 'bytes'>
```